In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from datetime import datetime, timedelta
from geopy.geocoders import Nominatim

In [2]:
data = pd.read_csv('velibs_export_2024-02-13_10:44.csv')
data


,identifiant_station,nom_station,station_en_fonctionnement,capacité_de_la_station,nombre_bornettes_libres,nombre_total_vélos_disponibles,vélos_mécaniques_disponibles,vélos_électriques_disponibles,borne_de_paiement_disponible,retour_vélib_possible,actualisation_de_la_donnée,coordonnées_géographiques,nom_communes_équipées,code_insee_communes_équipées
0,44015,Rouget de L'isle - Watteau,OUI,20.0,10.0,10.0,5.0,5.0,OUI,OUI,2024-02-12T12:59:40+01:00,"48.778192750803, 2.3963020229163",Vitry-sur-Seine,NaN
1,16107,Benjamin Godard - Victor Hugo,OUI,35.0,30.0,5.0,0.0,5.0,OUI,OUI,2024-02-12T12:54:31+01:00,"48.865983, 2.275725",Paris,NaN
2,9020,Toudouze - Clauzel,OUI,21.0,20.0,1.0,0.0,1.0,OUI,OUI,2024-02-12T12:59:10+01:00,"48.87929591733507, 2.3373600840568547",Paris,NaN
3,31104,Mairie de Rosny-sous-Bois,OUI,30.0,19.0,9.0,2.0,7.0,OUI,OUI,2024-02-12T13:01:04+01:00,"48.871256519012, 2.4865807592869",Rosny-sous-Bois,NaN
4,7002,Vaneau - Sèvres,OUI,35.0,16.0,17.0,13.0,4.0,OUI,OUI,2024-02-12T13:00:13+01:00,"48.848563233059, 2.3204218259346",Paris,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
165050,41212,Nungesser - Santé,OUI,22.0,21.0,1.0,0.0,1.0,OUI,OUI,2024-02-13T09:58:48+01:00,"48.852335, 2.448967",Fontenay-sous-Bois,NaN
165051,9104,Caumartin - Provence,OUI,22.0,1.0,33.0,17.0,16.0,OUI,OUI,2024-02-13T10:03:09+01:00,"48.874422773426545, 2.3284685611724854",Paris,NaN
165052,8004,Malesherbes - Place de la Madeleine,OUI,67.0,2.0,75.0,36.0,39.0,OUI,OUI,2024-02-13T10:02:43+01:00,"48.870406028483, 2.323243509808",Paris,NaN
165053,11014,Place de la Nation - Taillebourg,OUI,47.0,19.0,26.0,9.0,17.0,OUI,OUI,2024-02-13T10:01:02+01:00,"48.8488408, 2.3973043",Paris,NaN


In [3]:
data.columns

Index(['identifiant_station', 'nom_station', 'station_en_fonctionnement',
       'capacité_de_la_station', 'nombre_bornettes_libres',
       'nombre_total_vélos_disponibles', 'vélos_mécaniques_disponibles',
       'vélos_électriques_disponibles', 'borne_de_paiement_disponible',
       'retour_vélib_possible', 'actualisation_de_la_donnée',
       'coordonnées_géographiques', 'nom_communes_équipées',
       'code_insee_communes_équipées'],
      dtype='object')

In [4]:
df = data.rename(columns={
    'identifiant_station': 'ID',
    'nom_station': 'nom',
    'station_en_fonctionnement': 'status',
    'capacité_de_la_station': 'capacity',
    'nombre_bornettes_libres': 'availability_kiosk',
    'nombre_total_vélos_disponibles': 'availability_bike',
    'vélos_mécaniques_disponibles': 'mbikes_available',
    'vélos_électriques_disponibles': 'ebike_available',
    'borne_de_paiement_disponible': 'payment_possible',
    'retour_vélib_possible': 'return_available',
    'actualisation_de_la_donnée': 'update_time',
    'coordonnées_géographiques': 'coordinates',
    'nom_communes_équipées': 'communes',
    'code_insee_communes_équipées': 'Code_INSEE',
    'update_time': 'collection_time'
})

In [5]:
df.isna().any()

ID                    True
nom                   True
status                True
capacity              True
availability_kiosk    True
availability_bike     True
mbikes_available      True
ebike_available       True
payment_possible      True
return_available      True
update_time           True
coordinates           True
communes              True
Code_INSEE            True
dtype: bool

In [6]:
df = df.drop(['Code_INSEE','payment_possible'], axis=1)


In [7]:
df[df['status'].isna()]
df['ID'].unique()
df[df['ID'].apply(lambda x: str(x).isdigit())]['ID'].unique()
df = df[df['ID'].astype(str).str.isdigit()]

In [8]:
df = df.drop_duplicates()

In [9]:
df.isna().any()

ID                    False
nom                   False
status                False
capacity              False
availability_kiosk    False
availability_bike     False
mbikes_available      False
ebike_available       False
return_available      False
update_time           False
coordinates           False
communes              False
dtype: bool

In [10]:
df.dtypes

ID                     object
nom                    object
status                 object
capacity              float64
availability_kiosk    float64
availability_bike     float64
mbikes_available      float64
ebike_available       float64
return_available       object
update_time            object
coordinates            object
communes               object
dtype: object

In [11]:
df['status'].unique()

array(['OUI', 'NON'], dtype=object)

In [12]:
df['status'] = pd.Series(df['status']).map({'OUI': True, 'NON': False})
df['return_available'] = pd.Series(df['return_available']).map({'OUI': True, 'NON': False})
df['unavailable'] = (df['status'] == False) | (df['availability_bike'] == 0)

In [13]:
df['status'].unique()

array([ True, False])

In [14]:
df.isna().any()

ID                    False
nom                   False
status                False
capacity              False
availability_kiosk    False
availability_bike     False
mbikes_available      False
ebike_available       False
return_available      False
update_time           False
coordinates           False
communes              False
unavailable           False
dtype: bool

In [15]:
df.shape

(149310, 13)

In [16]:
df.dtypes

ID                     object
nom                    object
status                   bool
capacity              float64
availability_kiosk    float64
availability_bike     float64
mbikes_available      float64
ebike_available       float64
return_available         bool
update_time            object
coordinates            object
communes               object
unavailable              bool
dtype: object

In [17]:
total_stations= df['ID'].nunique()

In [18]:
unavailable_stations=df[(df['status'] == False)]['ID'].unique()
df[df['ID'].isin(unavailable_stations)]

,ID,nom,status,capacity,availability_kiosk,availability_bike,mbikes_available,ebike_available,return_available,update_time,coordinates,communes,unavailable
144,51005,Yves Farges - Route d'Enghein,False,0.0,0.0,0.0,0.0,0.0,False,2024-02-06T12:04:01+01:00,"48.95756799268, 2.2834496977978",Argenteuil,True
267,51004,Général Delambre - Prebuard,False,0.0,0.0,0.0,0.0,0.0,False,2024-02-08T07:18:58+01:00,"48.93870344444, 2.2234171606307",Argenteuil,True
515,8001,Petit Palais,False,46.0,46.0,0.0,0.0,0.0,False,2024-02-12T12:53:23+01:00,"48.8667692, 2.3157655",Paris,True
547,17201,Porte Maillot - Perreire,False,0.0,0.0,0.0,0.0,0.0,False,2023-09-11T16:39:27+02:00,"48.878021154637, 2.2842371277102",Paris,True
1164,16201,Porte Dauphine,False,0.0,0.0,0.0,0.0,0.0,False,2023-07-28T18:29:09+02:00,"48.871139344351, 2.2749781616017",Paris,True
1372,26010,Louis Armand - Bernard Palissy,False,0.0,0.0,0.0,0.0,0.0,False,2023-01-25T15:17:38+01:00,"48.916925546419, 2.3124207955211",Asnières-sur-Seine,True
1373,31201,Suzanne Masson,False,0.0,0.0,0.0,0.0,0.0,False,2024-01-15T08:20:05+01:00,"48.923306888013, 2.3847292397611",La Courneuve,True
1377,31202,Gabriel Péri - Jules Ferry,False,0.0,0.0,0.0,0.0,0.0,False,2024-01-19T11:21:26+01:00,"48.926182814311, 2.3912295762525",La Courneuve,True
1379,19204,Rampal - Belleville,False,0.0,0.0,0.0,0.0,0.0,False,2023-09-19T10:41:41+02:00,"48.873114661405, 2.3798409114805",Paris,True
4942,8001,Petit Palais,False,46.0,46.0,0.0,0.0,0.0,False,2024-02-12T13:53:51+01:00,"48.8667692, 2.3157655",Paris,True


In [19]:
df.groupby(['ID']).agg({'availability_bike':np.mean})

,availability_bike
ID,
10001,11.041379
10003,9.724832
10004,14.925764
10005,8.131250
10006,8.791469
...,...
92004,16.589286
92005,17.666667
92006,6.500000


In [20]:
df.groupby(['ID']).agg({'availability_bike':np.mean}).mean()

availability_bike    12.168655
dtype: float64

In [21]:
df = df.sort_values(by=['ID', 'update_time'])
grouped = df.groupby('ID')
df['unavailability_diff'] = grouped['unavailable'].diff()
df['unavailability_diff'] .unique()

array([nan, 0, 1, -1], dtype=object)

In [22]:
df

,ID,nom,status,capacity,availability_kiosk,availability_bike,mbikes_available,ebike_available,return_available,update_time,coordinates,communes,unavailable,unavailability_diff
1374,10001,René Boulanger - Lancry,True,62.0,49.0,8.0,4.0,4.0,True,2024-02-12T13:00:54+01:00,"48.8685433, 2.3600032",Paris,False,NaN
1647,10001,René Boulanger - Lancry,True,62.0,50.0,7.0,4.0,3.0,True,2024-02-12T13:00:54+01:00,"48.8685433, 2.3600032",Paris,False,0
2266,10001,René Boulanger - Lancry,True,62.0,51.0,6.0,4.0,2.0,True,2024-02-12T13:00:54+01:00,"48.8685433, 2.3600032",Paris,False,0
2774,10001,René Boulanger - Lancry,True,62.0,49.0,8.0,6.0,2.0,True,2024-02-12T13:00:54+01:00,"48.8685433, 2.3600032",Paris,False,0
2923,10001,René Boulanger - Lancry,True,62.0,50.0,7.0,5.0,2.0,True,2024-02-12T13:00:54+01:00,"48.8685433, 2.3600032",Paris,False,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
147515,92008,Hôtel de Ville de Nanterre,True,38.0,24.0,14.0,11.0,3.0,True,2024-02-13T07:57:58+01:00,"48.892371898250424, 2.205548956990242",Nanterre,False,0
148730,92008,Hôtel de Ville de Nanterre,True,38.0,23.0,15.0,12.0,3.0,True,2024-02-13T07:57:58+01:00,"48.892371898250424, 2.205548956990242",Nanterre,False,0
149223,92008,Hôtel de Ville de Nanterre,True,38.0,23.0,15.0,12.0,3.0,True,2024-02-13T09:00:56+01:00,"48.892371898250424, 2.205548956990242",Nanterre,False,0
149783,92008,Hôtel de Ville de Nanterre,True,38.0,24.0,14.0,12.0,2.0,True,2024-02-13T09:00:56+01:00,"48.892371898250424, 2.205548956990242",Nanterre,False,0


In [23]:
changed_stations = df[(df['availability_bike'] == 0) & (df['unavailability_diff'] != 0)]['ID'].unique()

len(changed_stations)

490

In [24]:
#df['ID'].value_counts() 

In [25]:
grouped = df.groupby('ID').agg(count=('ID', 'count'))

grouped = grouped.sort_values(by=['count'], ascending=False)
grouped[grouped['count']>1]

,count
ID,
8002,325
12151,319
11001,295
11048,269
43005,260
...,...
8025,22
18203,22
18134,22


In [26]:
#df[df['ID'].isin(grouped[grouped['count'] > 1].index)]

In [27]:
#df[df.groupby('ID')['ID'].transform('count') > 1]

In [28]:
#threshold = pd.Timedelta(minutes=30)

In [29]:
df['update_time'] = pd.to_datetime(df['update_time'])

In [30]:
df = df.sort_values(by=['update_time','nom'])
df['update_time'].min()

Timestamp('2017-12-07 12:18:58+0100', tz='UTC+01:00')

In [31]:
lower_date = pd.Timestamp('2024-02-11 00:00:00+0100', tz='UTC+01:00')

In [32]:
df = df[df['update_time'] >= lower_date]

In [33]:
df

,ID,nom,status,capacity,availability_kiosk,availability_bike,mbikes_available,ebike_available,return_available,update_time,coordinates,communes,unavailable,unavailability_diff
1381,15062,Convention - Lourmel,True,36.0,36.0,0.0,0.0,0.0,False,2024-02-12 07:55:23+01:00,"48.842055041656, 2.2859910875559",Paris,True,NaN
45,16138,Saint-Cloud - Hippodrome,True,30.0,29.0,0.0,0.0,0.0,True,2024-02-12 12:52:22+01:00,"48.85772148887421, 2.2549109905958176",Paris,True,NaN
1823,16138,Saint-Cloud - Hippodrome,True,30.0,28.0,1.0,0.0,1.0,True,2024-02-12 12:52:22+01:00,"48.85772148887421, 2.2549109905958176",Paris,False,-1
1199,15050,Dominique Pado - Croix Nivert,True,22.0,19.0,3.0,1.0,2.0,True,2024-02-12 12:52:25+01:00,"48.83648798164934, 2.2897250577807426",Paris,False,NaN
1799,15050,Dominique Pado - Croix Nivert,True,22.0,18.0,4.0,2.0,2.0,True,2024-02-12 12:52:25+01:00,"48.83648798164934, 2.2897250577807426",Paris,False,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
160931,92007,Pesaro - Préfecture,True,22.0,6.0,16.0,2.0,14.0,True,2024-02-13 10:03:17+01:00,"48.89577046284005, 2.223908342421055",Nanterre,False,0
162420,6028,Quatre Vents - Carrefour de l'Odéon,True,37.0,25.0,11.0,4.0,7.0,True,2024-02-13 10:03:17+01:00,"48.851751374458864, 2.338161695134053",Paris,False,0
162577,6028,Quatre Vents - Carrefour de l'Odéon,True,37.0,26.0,10.0,4.0,6.0,True,2024-02-13 10:03:17+01:00,"48.851751374458864, 2.338161695134053",Paris,False,0
162873,6028,Quatre Vents - Carrefour de l'Odéon,True,37.0,27.0,9.0,3.0,6.0,True,2024-02-13 10:03:17+01:00,"48.851751374458864, 2.338161695134053",Paris,False,0


In [34]:
df[df['ID']=='16138']

,ID,nom,status,capacity,availability_kiosk,availability_bike,mbikes_available,ebike_available,return_available,update_time,coordinates,communes,unavailable,unavailability_diff
45,16138,Saint-Cloud - Hippodrome,True,30.0,29.0,0.0,0.0,0.0,True,2024-02-12 12:52:22+01:00,"48.85772148887421, 2.2549109905958176",Paris,True,NaN
1823,16138,Saint-Cloud - Hippodrome,True,30.0,28.0,1.0,0.0,1.0,True,2024-02-12 12:52:22+01:00,"48.85772148887421, 2.2549109905958176",Paris,False,-1
4430,16138,Saint-Cloud - Hippodrome,True,30.0,28.0,1.0,0.0,1.0,True,2024-02-12 13:52:51+01:00,"48.85772148887421, 2.2549109905958176",Paris,False,0
8702,16138,Saint-Cloud - Hippodrome,True,30.0,29.0,0.0,0.0,0.0,True,2024-02-12 13:52:51+01:00,"48.85772148887421, 2.2549109905958176",Paris,True,1
11187,16138,Saint-Cloud - Hippodrome,True,30.0,28.0,1.0,1.0,0.0,True,2024-02-12 13:52:51+01:00,"48.85772148887421, 2.2549109905958176",Paris,False,-1
12892,16138,Saint-Cloud - Hippodrome,True,30.0,27.0,2.0,2.0,0.0,True,2024-02-12 15:01:35+01:00,"48.85772148887421, 2.2549109905958176",Paris,False,0
17602,16138,Saint-Cloud - Hippodrome,True,30.0,28.0,1.0,1.0,0.0,True,2024-02-12 15:01:35+01:00,"48.85772148887421, 2.2549109905958176",Paris,False,0
21360,16138,Saint-Cloud - Hippodrome,True,30.0,28.0,1.0,1.0,0.0,True,2024-02-12 15:57:24+01:00,"48.85772148887421, 2.2549109905958176",Paris,False,0
25914,16138,Saint-Cloud - Hippodrome,True,30.0,27.0,2.0,1.0,1.0,True,2024-02-12 15:57:24+01:00,"48.85772148887421, 2.2549109905958176",Paris,False,0
26891,16138,Saint-Cloud - Hippodrome,True,30.0,29.0,0.0,0.0,0.0,True,2024-02-12 15:57:24+01:00,"48.85772148887421, 2.2549109905958176",Paris,True,1


In [35]:
df.dtypes

ID                      object
nom                     object
status                    bool
capacity               float64
availability_kiosk     float64
availability_bike      float64
mbikes_available       float64
ebike_available        float64
return_available          bool
update_time             object
coordinates             object
communes                object
unavailable               bool
unavailability_diff     object
dtype: object

In [36]:
df['unavailability_diff'] = df['unavailability_diff'].fillna(0)
df['unavailability_diff'] = df['unavailability_diff'].astype(int)

In [37]:
def calculate_unavailable_durations(df):
    result_data = []
    grouped = df.groupby('ID')
    
    for group_name, group_df in grouped:
        sorted_group_df = group_df.sort_values('update_time')
        
        unavailable_time = None
        for index, row in sorted_group_df.iterrows():
            if row['unavailable'] == True and row['unavailability_diff'] == 1:
                unavailable_time = pd.to_datetime(row['update_time'])
            elif row['unavailability_diff'] == -1 and unavailable_time is not None:
                available_time = pd.to_datetime(row['update_time'])
                unavailable_duration = available_time - unavailable_time
                if unavailable_duration > pd.Timedelta(0):
                    result_data.append({
                        'ID': group_name, 
                        'nom': group_df['nom'].iloc[0], 
                        'coordinates': group_df['coordinates'].iloc[0], 
                        'communes': group_df['communes'].iloc[0], 
                        'capacity': group_df['capacity'].iloc[0], 
                        'available_time': available_time,
                        'unavailable_time': unavailable_time,
                        'unavailable_duration': unavailable_duration
                    })
                unavailable_time = None

    result_df = pd.DataFrame(result_data)
    return result_df


In [38]:
durations = calculate_unavailable_durations(df)
durations = durations[durations['unavailable_duration']>'0 days 0:03:00']

In [39]:
durations

,ID,nom,coordinates,communes,capacity,available_time,unavailable_time,unavailable_duration
0,10009,Mairie du 10ème,"48.872089449907, 2.3575825989246",Paris,19.0,2024-02-12 15:53:11+01:00,2024-02-12 14:51:43+01:00,0 days 01:01:28
1,10009,Mairie du 10ème,"48.872089449907, 2.3575825989246",Paris,19.0,2024-02-12 17:00:10+01:00,2024-02-12 15:53:11+01:00,0 days 01:06:59
2,10016,Récollets - Square Villemin,"48.875034655883944, 2.359801238597928",Paris,17.0,2024-02-12 17:56:17+01:00,2024-02-12 16:59:52+01:00,0 days 00:56:25
3,10016,Récollets - Square Villemin,"48.875034655883944, 2.359801238597928",Paris,17.0,2024-02-12 20:57:50+01:00,2024-02-12 18:56:51+01:00,0 days 02:00:59
4,10016,Récollets - Square Villemin,"48.875034655883944, 2.359801238597928",Paris,17.0,2024-02-12 22:58:43+01:00,2024-02-12 21:58:16+01:00,0 days 01:00:27
...,...,...,...,...,...,...,...,...
530,9114,Faubourg Poissonnière - Dunkerque,"48.880883445404436, 2.349934279918671",Paris,30.0,2024-02-13 08:59:38+01:00,2024-02-13 08:01:31+01:00,0 days 00:58:07
531,9115,Milton - Manuel,"48.878249, 2.341181",Paris,22.0,2024-02-12 13:56:55+01:00,2024-02-12 13:00:00+01:00,0 days 00:56:55
532,9117,Bellefond - Maubeuge,"48.878255397645, 2.3452642187476",Paris,31.0,2024-02-12 20:00:50+01:00,2024-02-12 19:02:38+01:00,0 days 00:58:12
533,9117,Bellefond - Maubeuge,"48.878255397645, 2.3452642187476",Paris,31.0,2024-02-13 08:56:57+01:00,2024-02-13 08:02:27+01:00,0 days 00:54:30


In [40]:
df[df['ID'] == '9117'].tail(20)

,ID,nom,status,capacity,availability_kiosk,availability_bike,mbikes_available,ebike_available,return_available,update_time,coordinates,communes,unavailable,unavailability_diff
133542,9117,Bellefond - Maubeuge,True,31.0,24.0,7.0,3.0,4.0,True,2024-02-13 07:00:49+01:00,"48.878255397645, 2.3452642187476",Paris,False,0
134564,9117,Bellefond - Maubeuge,True,31.0,25.0,6.0,3.0,3.0,True,2024-02-13 07:00:49+01:00,"48.878255397645, 2.3452642187476",Paris,False,0
135139,9117,Bellefond - Maubeuge,True,31.0,25.0,6.0,3.0,3.0,True,2024-02-13 08:02:27+01:00,"48.878255397645, 2.3452642187476",Paris,False,0
136529,9117,Bellefond - Maubeuge,True,31.0,26.0,5.0,3.0,2.0,True,2024-02-13 08:02:27+01:00,"48.878255397645, 2.3452642187476",Paris,False,0
137123,9117,Bellefond - Maubeuge,True,31.0,27.0,4.0,2.0,2.0,True,2024-02-13 08:02:27+01:00,"48.878255397645, 2.3452642187476",Paris,False,0
137468,9117,Bellefond - Maubeuge,True,31.0,27.0,3.0,1.0,2.0,True,2024-02-13 08:02:27+01:00,"48.878255397645, 2.3452642187476",Paris,False,0
137826,9117,Bellefond - Maubeuge,True,31.0,26.0,4.0,2.0,2.0,True,2024-02-13 08:02:27+01:00,"48.878255397645, 2.3452642187476",Paris,False,0
138596,9117,Bellefond - Maubeuge,True,31.0,26.0,5.0,2.0,3.0,True,2024-02-13 08:02:27+01:00,"48.878255397645, 2.3452642187476",Paris,False,0
138799,9117,Bellefond - Maubeuge,True,31.0,25.0,5.0,2.0,3.0,True,2024-02-13 08:02:27+01:00,"48.878255397645, 2.3452642187476",Paris,False,0
139426,9117,Bellefond - Maubeuge,True,31.0,25.0,5.0,3.0,2.0,True,2024-02-13 08:02:27+01:00,"48.878255397645, 2.3452642187476",Paris,False,0


In [41]:
durations[durations['ID'] == '9117']

,ID,nom,coordinates,communes,capacity,available_time,unavailable_time,unavailable_duration
532,9117,Bellefond - Maubeuge,"48.878255397645, 2.3452642187476",Paris,31.0,2024-02-12 20:00:50+01:00,2024-02-12 19:02:38+01:00,0 days 00:58:12
533,9117,Bellefond - Maubeuge,"48.878255397645, 2.3452642187476",Paris,31.0,2024-02-13 08:56:57+01:00,2024-02-13 08:02:27+01:00,0 days 00:54:30


In [42]:
problematic_stations = durations[durations['unavailable_duration']>'0 days 0:30:00']['nom'].unique()

In [43]:
problematic_stations

array(['Mairie du 10ème', 'Récollets - Square Villemin',
       "Gare de l'Est - Verdun", 'Jemmapes - Ecluses Saint-Martin',
       'Dunkerque - Alsace', 'Dunkerque - Rocroy',
       'Claude Vellefaux - Hôpital Saint-Louis', 'Quai de Valmy',
       'Gare du Nord - La Chapelle', 'Louis Blanc - Jemmapes',
       'Gare du Nord - Denain', "Gare de l'Est - Faubourg-Saint-Martin",
       'Claude Vellefaux - Sambre et Meuse', 'Strasbourg - Saint-Denis',
       'Jean-Pierre Timbaud - Vaucouleurs',
       'Parmentier - Fontaine au Roi', 'Parmentier - République',
       'Cossonnerie - Sébastopol', 'Gare de Reuilly - Picpus',
       'Gare de Bercy', 'Terroirs de France - Lheureux', 'Niger - Soult',
       'Nationale - Vincent Auriol', 'Butte aux Cailles - Esperance',
       'Ivry - Simone Weil', 'Halle Freyssinet - Parvis Alan Turing',
       'Croulebarde - Corvisart', 'Hôpital Pitié Salpêtrière',
       'Bobillot - Paulin Mery', 'Brillat-Savarin - Küss',
       'Moulin de la Pointe', 'Nationale

In [96]:
available_stations = df[~df['ID'].isin(unavailable_stations)]['ID'].unique()

In [97]:
(len(problematic_stations)+len(unavailable_stations))/len(available_stations)

0.23549965059399022

In [98]:
(len(available_stations)+len(unavailable_stations))

1440

In [99]:
df['ID'].nunique()

1432

In [48]:
durations

,ID,nom,coordinates,communes,capacity,available_time,unavailable_time,unavailable_duration
0,10009,Mairie du 10ème,"48.872089449907, 2.3575825989246",Paris,19.0,2024-02-12 15:53:11+01:00,2024-02-12 14:51:43+01:00,0 days 01:01:28
1,10009,Mairie du 10ème,"48.872089449907, 2.3575825989246",Paris,19.0,2024-02-12 17:00:10+01:00,2024-02-12 15:53:11+01:00,0 days 01:06:59
2,10016,Récollets - Square Villemin,"48.875034655883944, 2.359801238597928",Paris,17.0,2024-02-12 17:56:17+01:00,2024-02-12 16:59:52+01:00,0 days 00:56:25
3,10016,Récollets - Square Villemin,"48.875034655883944, 2.359801238597928",Paris,17.0,2024-02-12 20:57:50+01:00,2024-02-12 18:56:51+01:00,0 days 02:00:59
4,10016,Récollets - Square Villemin,"48.875034655883944, 2.359801238597928",Paris,17.0,2024-02-12 22:58:43+01:00,2024-02-12 21:58:16+01:00,0 days 01:00:27
...,...,...,...,...,...,...,...,...
530,9114,Faubourg Poissonnière - Dunkerque,"48.880883445404436, 2.349934279918671",Paris,30.0,2024-02-13 08:59:38+01:00,2024-02-13 08:01:31+01:00,0 days 00:58:07
531,9115,Milton - Manuel,"48.878249, 2.341181",Paris,22.0,2024-02-12 13:56:55+01:00,2024-02-12 13:00:00+01:00,0 days 00:56:55
532,9117,Bellefond - Maubeuge,"48.878255397645, 2.3452642187476",Paris,31.0,2024-02-12 20:00:50+01:00,2024-02-12 19:02:38+01:00,0 days 00:58:12
533,9117,Bellefond - Maubeuge,"48.878255397645, 2.3452642187476",Paris,31.0,2024-02-13 08:56:57+01:00,2024-02-13 08:02:27+01:00,0 days 00:54:30


In [49]:
mean_availibility = df.groupby(['ID', 'nom', 'coordinates', 'communes','capacity']).agg({'availability_bike': 'mean'}).reset_index()

In [50]:
total_unavailability = durations.groupby(['ID', 'nom', 'coordinates', 'communes','capacity']).agg({'unavailable_duration': 'sum'}).reset_index()
total_unavailability = pd.merge(total_unavailability, (durations.groupby('ID').agg({'unavailable_duration': 'count'}).reset_index()), on='ID', how='inner')
total_unavailability.rename(columns={'unavailable_duration_x': 'unavailable_duration','unavailable_duration_y': 'unavailable_count'}, inplace=True)
total_unavailability

,ID,nom,coordinates,communes,capacity,unavailable_duration,unavailable_count
0,10009,Mairie du 10ème,"48.872089449907, 2.3575825989246",Paris,19.0,0 days 02:08:27,2
1,10016,Récollets - Square Villemin,"48.875034655883944, 2.359801238597928",Paris,17.0,0 days 03:57:51,3
2,10023,Gare de l'Est - Verdun,"48.876156054271, 2.3609149083206",Paris,14.0,0 days 02:58:50,3
3,10025,Jemmapes - Ecluses Saint-Martin,"48.87748519186726, 2.3660067468881607",Paris,15.0,0 days 00:56:24,1
4,10027,Dunkerque - Alsace,"48.879355, 2.358379",Paris,20.0,0 days 00:58:56,1
...,...,...,...,...,...,...,...
324,9039,Chaussée d'Antin - Haussmann,"48.87243, 2.33359",Paris,24.0,0 days 02:01:34,1
325,9114,Faubourg Poissonnière - Dunkerque,"48.880883445404436, 2.349934279918671",Paris,30.0,0 days 02:56:51,3
326,9115,Milton - Manuel,"48.878249, 2.341181",Paris,22.0,0 days 00:56:55,1
327,9117,Bellefond - Maubeuge,"48.878255397645, 2.3452642187476",Paris,31.0,0 days 01:52:42,2


In [51]:
mean_availibility

,ID,nom,coordinates,communes,capacity,availability_bike
0,10001,René Boulanger - Lancry,"48.8685433, 2.3600032",Paris,62.0,11.041379
1,10003,Hauteville - Bonne Nouvelle,"48.87077436024, 2.3496496368327",Paris,17.0,9.724832
2,10004,Metz - Faubourg Saint-Denis,"48.870893, 2.353521",Paris,48.0,14.925764
3,10005,Petites Ecuries - Faubourg Poissonnière,"48.87397217237368, 2.348388757504776",Paris,17.0,8.131250
4,10006,Petites Ecuries - Faubourg Saint-Denis,"48.87287120589831, 2.3542150855064397",Paris,28.0,8.791469
...,...,...,...,...,...,...
1427,92004,Nanterre - Université,"48.901365, 2.212693",Nanterre,20.0,16.589286
1428,92005,Gare de Nanterre Ville,"48.8950295, 2.1957932",Nanterre,43.0,17.666667
1429,92006,Place Nelson Mandela,"48.897874318180556, 2.2182820737361912",Nanterre,30.0,6.500000
1430,92007,Pesaro - Préfecture,"48.89577046284005, 2.223908342421055",Nanterre,22.0,5.896552


In [85]:
availability = pd.merge(mean_availibility, total_unavailability, on=['ID', 'nom', 'coordinates', 'communes', 'capacity'], how='left')
availability['unavailable_duration'] = availability['unavailable_duration'].fillna(pd.Timedelta(seconds=0))
availability['unavailable_count'] = availability['unavailable_count'].fillna(0)

In [86]:
availability

,ID,nom,coordinates,communes,capacity,availability_bike,unavailable_duration,unavailable_count
0,10001,René Boulanger - Lancry,"48.8685433, 2.3600032",Paris,62.0,11.041379,0 days 00:00:00,0.0
1,10003,Hauteville - Bonne Nouvelle,"48.87077436024, 2.3496496368327",Paris,17.0,9.724832,0 days 00:00:00,0.0
2,10004,Metz - Faubourg Saint-Denis,"48.870893, 2.353521",Paris,48.0,14.925764,0 days 00:00:00,0.0
3,10005,Petites Ecuries - Faubourg Poissonnière,"48.87397217237368, 2.348388757504776",Paris,17.0,8.131250,0 days 00:00:00,0.0
4,10006,Petites Ecuries - Faubourg Saint-Denis,"48.87287120589831, 2.3542150855064397",Paris,28.0,8.791469,0 days 00:00:00,0.0
...,...,...,...,...,...,...,...,...
1427,92004,Nanterre - Université,"48.901365, 2.212693",Nanterre,20.0,16.589286,0 days 00:00:00,0.0
1428,92005,Gare de Nanterre Ville,"48.8950295, 2.1957932",Nanterre,43.0,17.666667,0 days 00:00:00,0.0
1429,92006,Place Nelson Mandela,"48.897874318180556, 2.2182820737361912",Nanterre,30.0,6.500000,0 days 00:00:00,0.0
1430,92007,Pesaro - Préfecture,"48.89577046284005, 2.223908342421055",Nanterre,22.0,5.896552,0 days 01:00:26,1.0


In [87]:
availability['capacity'] = availability['capacity'].astype(int)
availability['unavailable_count'] = availability['unavailable_count'].astype(int)
availability.dtypes

ID                               object
nom                              object
coordinates                      object
communes                         object
capacity                          int64
availability_bike               float64
unavailable_duration    timedelta64[ns]
unavailable_count                 int64
dtype: object

In [88]:
availability['unavailable_duration'] = np.ceil(availability['unavailable_duration'].dt.total_seconds() / 60).astype(int)


In [89]:
availability

,ID,nom,coordinates,communes,capacity,availability_bike,unavailable_duration,unavailable_count
0,10001,René Boulanger - Lancry,"48.8685433, 2.3600032",Paris,62,11.041379,0,0
1,10003,Hauteville - Bonne Nouvelle,"48.87077436024, 2.3496496368327",Paris,17,9.724832,0,0
2,10004,Metz - Faubourg Saint-Denis,"48.870893, 2.353521",Paris,48,14.925764,0,0
3,10005,Petites Ecuries - Faubourg Poissonnière,"48.87397217237368, 2.348388757504776",Paris,17,8.131250,0,0
4,10006,Petites Ecuries - Faubourg Saint-Denis,"48.87287120589831, 2.3542150855064397",Paris,28,8.791469,0,0
...,...,...,...,...,...,...,...,...
1427,92004,Nanterre - Université,"48.901365, 2.212693",Nanterre,20,16.589286,0,0
1428,92005,Gare de Nanterre Ville,"48.8950295, 2.1957932",Nanterre,43,17.666667,0,0
1429,92006,Place Nelson Mandela,"48.897874318180556, 2.2182820737361912",Nanterre,30,6.500000,0,0
1430,92007,Pesaro - Préfecture,"48.89577046284005, 2.223908342421055",Nanterre,22,5.896552,61,1


In [57]:
geolocator = Nominatim(user_agent="postcode_extractor")

In [58]:
%%time
geolocator.reverse('48.87287120589831, 2.3542150855064397').raw.get('address').get('suburb')

CPU times: user 8.73 ms, sys: 7.54 ms, total: 16.3 ms
Wall time: 213 ms


'Paris 10e Arrondissement'

In [60]:
paris_stations = availability[availability['communes'] == 'Paris'][['nom', 'coordinates']]
paris_stations

,nom,coordinates
0,René Boulanger - Lancry,"48.8685433, 2.3600032"
1,Hauteville - Bonne Nouvelle,"48.87077436024, 2.3496496368327"
2,Metz - Faubourg Saint-Denis,"48.870893, 2.353521"
3,Petites Ecuries - Faubourg Poissonnière,"48.87397217237368, 2.348388757504776"
4,Petites Ecuries - Faubourg Saint-Denis,"48.87287120589831, 2.3542150855064397"
...,...,...
1419,Faubourg Poissonnière - Dunkerque,"48.880883445404436, 2.349934279918671"
1420,Milton - Manuel,"48.878249, 2.341181"
1421,Victoire - Chaussée d'Antin,"48.875065791004396, 2.3316738009452824"
1422,Bellefond - Maubeuge,"48.878255397645, 2.3452642187476"


In [63]:
def extract_arr(coordinates):
    location = geolocator.reverse(coordinates)
    postcode = location.raw.get('address', {}).get('postcode')
    if postcode:
        arrondissement = 'Paris ' + str(postcode)[-2:] + 'e Arrondissement'
        return arrondissement
    else:
        return None

extract_arr(paris_stations['coordinates'][0])

'Paris 10e Arrondissement'

In [64]:
geolocator.reverse('48.8242469, 2.3186306').raw.get('address')

{'house_number': '8',
 'road': 'Avenue de la Porte de Châtillon',
 'city_block': 'Quartier de Plaisance',
 'neighbourhood': 'Paris 14e Arrondissement',
 'suburb': 'Paris',
 'city_district': 'Paris',
 'city': 'Paris',
 'ISO3166-2-lvl6': 'FR-75',
 'state': 'Île-de-France',
 'ISO3166-2-lvl4': 'FR-IDF',
 'region': 'France métropolitaine',
 'postcode': '75014',
 'country': 'France',
 'country_code': 'fr'}

In [65]:
def extract_postcode(coordinates):
    location = geolocator.reverse(coordinates)
    if location:
        address = location.raw.get('address')
        if address:
            postcode = address.get('postcode')
            return postcode
    return None

extract_postcode('48.8685433, 2.3600032')

'75010'

In [66]:
%%time
paris_stations['arrondissement']=paris_stations['coordinates'].apply(extract_arr)

CPU times: user 2.67 s, sys: 864 ms, total: 3.54 s
Wall time: 8min 12s


In [79]:
paris_stations.loc[paris_stations['nom'] == 'Gare de Lyon - Chalon', 'arrondissement'] = 'Paris 12e Arrondissement'
paris_stations.loc[paris_stations['nom'] == 'Jourdan - Cité Universitaire', 'arrondissement'] = 'Paris 14e Arrondissement'

In [80]:
display(paris_stations)
paris_stations['arrondissement'].unique()

,nom,coordinates,arrondissement
0,René Boulanger - Lancry,"48.8685433, 2.3600032",Paris 10e Arrondissement
1,Hauteville - Bonne Nouvelle,"48.87077436024, 2.3496496368327",Paris 10e Arrondissement
2,Metz - Faubourg Saint-Denis,"48.870893, 2.353521",Paris 10e Arrondissement
3,Petites Ecuries - Faubourg Poissonnière,"48.87397217237368, 2.348388757504776",Paris 10e Arrondissement
4,Petites Ecuries - Faubourg Saint-Denis,"48.87287120589831, 2.3542150855064397",Paris 10e Arrondissement
...,...,...,...
1419,Faubourg Poissonnière - Dunkerque,"48.880883445404436, 2.349934279918671",Paris 10e Arrondissement
1420,Milton - Manuel,"48.878249, 2.341181",Paris 09e Arrondissement
1421,Victoire - Chaussée d'Antin,"48.875065791004396, 2.3316738009452824",Paris 09e Arrondissement
1422,Bellefond - Maubeuge,"48.878255397645, 2.3452642187476",Paris 09e Arrondissement


array(['Paris 10e Arrondissement', 'Paris 01e Arrondissement',
       'Paris 02e Arrondissement', 'Paris 11e Arrondissement',
       'Paris 20e Arrondissement', 'Paris 12e Arrondissement',
       'Paris 13e Arrondissement', 'Paris 14e Arrondissement',
       'Paris 15e Arrondissement', 'Paris 16e Arrondissement',
       'Paris 17e Arrondissement', 'Paris 08e Arrondissement',
       'Paris 18e Arrondissement', 'Paris 19e Arrondissement',
       'Paris 03e Arrondissement', 'Paris 04e Arrondissement',
       'Paris 05e Arrondissement', 'Paris 06e Arrondissement',
       'Paris 07e Arrondissement', 'Paris 09e Arrondissement'],
      dtype=object)

In [90]:
availability.loc[availability['communes'] == 'Paris', 'communes'] = paris_stations['arrondissement']

In [91]:
availability.loc[availability['communes'] == 'Paris', 'communes']

Series([], Name: communes, dtype: object)

In [92]:
availability['communes'].unique()

array(['Paris 10e Arrondissement', 'Paris 01e Arrondissement',
       'Paris 02e Arrondissement', 'Paris 11e Arrondissement',
       'Paris 20e Arrondissement', 'Paris 12e Arrondissement',
       'Saint-Mandé', 'Paris 13e Arrondissement', 'Ivry-sur-Seine',
       'Paris 14e Arrondissement', 'Malakoff', 'Montrouge',
       'Paris 15e Arrondissement', 'Paris 16e Arrondissement',
       'Paris 17e Arrondissement', 'Paris 08e Arrondissement',
       'Paris 18e Arrondissement', 'Paris 19e Arrondissement', 'Pantin',
       'Boulogne-Billancourt', 'Clichy', 'Issy-les-Moulineaux', 'Clamart',
       'Suresnes', 'Vanves', 'Meudon', 'Neuilly-sur-Seine', 'Saint-Cloud',
       'Bagneux', 'Gennevilliers', 'La Garenne-Colombes',
       'Fontenay-aux-Roses', 'Bois-Colombes', 'Levallois-Perret',
       'Bourg-la-Reine', 'Sèvres', 'Châtillon', 'Sceaux', 'Chaville',
       'Villeneuve-la-Garenne', 'Courbevoie', "Ville-d'Avray",
       'Rueil-Malmaison', 'Asnières-sur-Seine', 'Colombes', 'Puteaux',
      

In [93]:
availability.to_csv('availability.csv', index=False)

In [127]:
availability.to_csv('availability.csv', index=False)